In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\jaemin\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
####한국어 형태소 분석기####
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능
okt = Okt()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
df_18=pd.read_csv('./원본데이터/news18.csv',encoding='cp949')
df_19=pd.read_csv('./원본데이터/news19.csv',encoding='cp949')
df_20=pd.read_csv('./원본데이터/news20.csv',encoding='cp949')
df_21=pd.read_csv('./원본데이터/news21.csv',encoding='cp949')

In [6]:
df_18_price=pd.read_excel('./원본데이터/18년도 가격데이터.xlsx')
df_19_price=pd.read_excel('./원본데이터/19년도 가격데이터.xlsx')
df_20_price=pd.read_excel('./원본데이터/20년도 가격데이터.xlsx')
df_21_price=pd.read_excel('./원본데이터/21년도 가격데이터.xlsx')


In [7]:
#첫번째 행 삭제
df_18_price.drop(0,axis=0,inplace=True)
df_19_price.drop(0,axis=0,inplace=True)
df_20_price.drop(0,axis=0,inplace=True)
df_21_price.drop(0,axis=0,inplace=True)

In [8]:
df_price_all=pd.concat([df_18_price,df_19_price,df_20_price,df_21_price])

In [24]:
df_price_all[['업체코드','업체명','도축일','도체번호','경락단가']].tail()

,업체코드,업체명,도축일,도체번호,경락단가
50292,10466,도드람양돈협동조합,20211231,4239,4189.0
50293,10466,도드람양돈협동조합,20211231,4240,4269.0
50294,10466,도드람양돈협동조합,20211231,4241,4099.0
50295,10466,도드람양돈협동조합,20211231,4242,4329.0
50296,10466,도드람양돈협동조합,20211231,4243,4399.0


### 돼지고기 가격시간 datetime으로 변형

In [10]:
# df_price_all['도축일']=pd.to_datetime(df_price_all['도축일'])

In [11]:
df_price_all.head()

,업체코드,업체명,농가코드,농가명,이력번호,도축일,도체번호,판정일,판정구분,성별,생체중량,중량,등지방,1차등급,최종등급,경락단가,경락금액,삼겹비율,전체,Unnamed: 19,Unnamed: 20,등지방.1,Unnamed: 22,Unnamed: 23,삼겹,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,목심,Unnamed: 33,Unnamed: 34,갈비,Unnamed: 36,Unnamed: 37,등갈비\n무게,등심,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,안심\n무게,앞다리살,Unnamed: 46,Unnamed: 47,뒷다리살,Unnamed: 49,Unnamed: 50,Unnamed: 51,항정살\n무게,등심덧살\n무게,갈매기살\n무게,총 금액,Unnamed: 18,Unnamed: 21,Unnamed: 24,Unnamed: 32,Unnamed: 35,Unnamed: 39,Unnamed: 44,Unnamed: 45,Unnamed: 48
1,10466,도드람양돈협동조합,32816,엠파크(박광욱),140136200128,20180102,NaN,20180102,1,거세,116.098,88,24,1+,3,3622.0,318736.0,13.681818,63.5,32.29,23.12,30,25,25,12.04,9.9,5.3,27.25,2.54,13.39,54.47,40.51,4.44,3.8,69,3.64,3.62,61.5,0.86,5.34,NaN,5.88,65.51,35.19,0.98,8.08,7.74,60.51,16.86,NaN,15.68,69.84,0.48,0.38,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10466,도드람양돈협동조합,32816,엠파크(박광욱),140136200128,20180102,NaN,20180102,1,암,116.098,89,20,1+,3,3755.0,334195.0,14.089888,63.7,31.74,23.62,29,24,25,12.54,10.04,5.26,27.48,2.52,13.29,53.93,40.68,4.46,3.76,69.1,3.72,3.68,61.38,0.86,5.4,NaN,6,65.8,34.22,0.96,8.18,7.86,59.9,16.92,NaN,15.68,69.84,0.52,0.42,0.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10466,도드람양돈협동조합,32816,엠파크(박광욱),140136200128,20180102,NaN,20180102,1,거세,116.098,86,24,1+,3,3700.0,318200.0,13.674419,64.7,30.59,22.8,33,27,31,11.76,9.56,4.86,28.02,2.34,12.98,53.79,40.69,4.44,3.68,70.49,3.54,3.5,62.54,0.84,5.96,NaN,6.56,67.91,31.98,0.96,7.96,7.66,60.55,17.08,NaN,16.18,71.24,0.48,0.38,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10466,도드람양돈협동조합,32816,엠파크(박광욱),140136200128,20180102,NaN,20180102,1,거세,116.098,86,22,1+,3,3733.0,321038.0,14.139535,64.6,30.71,22.1,29,24,26,12.16,9.84,4.86,26.69,2.32,12.88,54.82,39.65,4.32,3.68,70.02,3.58,3.56,61.76,0.84,5.38,NaN,5.92,66.8,32.59,0.92,7.98,7.68,60.63,16.36,NaN,15.2,70.2,0.5,0.38,0.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10466,도드람양돈협동조합,32816,엠파크(박광욱),140136200128,20180102,NaN,20180102,1,거세,116.098,87,36,2,2,2199.0,191313.0,14.413793,57.1,39.03,28.8,41,36,40,12.54,9.04,6.18,31.58,3.4,17.27,47.21,48.83,4,3.22,62.96,3.38,3.34,57.73,0.86,4.72,NaN,5.24,54.86,45.24,0.84,7.48,6.84,54.06,15.08,NaN,13.4,64.35,0.48,0.38,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# df_price_all['datetime']=df_price_all['도축일'].dt.strftime('%Y-%m-%d')

# df_price_all['datetime']=pd.to_datetime(df_price_all['datetime'])

### 도축일 일별 기준 groupby 평균하기

In [27]:
df_price_all_2column=df_price_all[['도축일',"경락단가"]]

df_price_all_2column['도축일']=pd.to_datetime(df_price_all_2column['도축일'])

df_price_all_2column.head()

C:\Users\jaemin\AppData\Local\Temp\ipykernel_1992\1319833842.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_price_all_2column['도축일']=pd.to_datetime(df_price_all_2column['도축일'])


,도축일,경락단가
1,2018-01-02,3622.0
2,2018-01-02,3755.0
3,2018-01-02,3700.0
4,2018-01-02,3733.0
5,2018-01-02,2199.0


In [28]:
df_price_all_2column['경락단가_평균'] = df_price_all_2column.groupby(['도축일'])['경락단가'].transform('mean')

df_price_all_2column.head()

df_price_all_2column=df_price_all_2column.groupby(df_price_all_2column['도축일']).mean()

df_price_all_2column.reset_index(inplace=True)

new_df_price_all_2column=df_price_all_2column.iloc[238:,]

new_df_price_all_2column.reset_index(drop=True, inplace=True)

C:\Users\jaemin\AppData\Local\Temp\ipykernel_1992\2410941435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_price_all_2column['경락단가_평균'] = df_price_all_2column.groupby(['도축일'])['경락단가'].transform('mean')


In [31]:
new_df_price_all_2column

,도축일,경락단가,경락단가_평균
0,2018-01-02,3750.366093,3750.366093
1,2018-01-03,3547.574932,3547.574932
2,2018-01-04,3696.060897,3696.060897
3,2018-01-05,3770.156364,3770.156364
4,2018-01-08,3809.974684,3809.974684
5,2018-01-09,3723.371930,3723.371930
6,2018-01-10,3694.435484,3694.435484
7,2018-01-11,3740.687243,3740.687243
8,2018-01-12,3748.212245,3748.212245
9,2018-01-15,3800.211009,3800.211009


### 주별로 groupby해서 평균내기

In [41]:
week = [g.reset_index() for n, g in new_df_price_all_2column.set_index('도축일').groupby(pd.Grouper(freq='W'))]

In [42]:
week[0]

,도축일,경락단가,경락단가_평균
0,2018-01-02,3750.366093,3750.366093
1,2018-01-03,3547.574932,3547.574932
2,2018-01-04,3696.060897,3696.060897
3,2018-01-05,3770.156364,3770.156364


In [43]:
new_mean=new_df_price_all_2column.set_index('도축일').groupby(pd.Grouper(freq='W')).mean()

In [44]:
new_mean=new_mean.reset_index()

In [45]:
#결측치 바로 위에 값하고 동일하게 처리
new_mean=new_mean.fillna(method='ffill')

In [46]:
new_mean['label']=0

In [47]:
new_mean[['도축일','경락단가_평균','new_label']].iloc[1:10]

KeyError: "['new_label'] not in index"

### 전 주와 비교하여 감소:2, 유지:0, 증가:1

In [48]:
label_list=[]
for i in range(len(new_mean)):
    if(i!=0):
        if(new_mean['경락단가'][i-1] > new_mean['경락단가'][i]):
            new_mean['label'][i]=2
        elif (new_mean['경락단가'][i-1] < new_mean['경락단가'][i]):
            new_mean['label'][i]=1
        elif new_mean['경락단가'][i-1]==new_mean['경락단가'][i]:              
            new_mean['label'][i]=0
    else:
        new_mean['label'][i]=0

C:\Users\jaemin\AppData\Local\Temp\ipykernel_1992\1809203011.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mean['label'][i]=0
C:\Users\jaemin\AppData\Local\Temp\ipykernel_1992\1809203011.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mean['label'][i]=1
C:\Users\jaemin\AppData\Local\Temp\ipykernel_1992\1809203011.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mean['label'][i]=2
C:\Users\jaemin\AppData\Local\

In [52]:
new_mean[['도축일','경락단가_평균','label','가격등락비율']].iloc[1:10]

KeyError: "['가격등락비율'] not in index"

In [53]:
new_mean.head()

,도축일,경락단가,경락단가_평균,label
0,2018-01-07,3691.039572,3691.039572,0
1,2018-01-14,3743.336317,3743.336317,1
2,2018-01-21,3743.359986,3743.359986,1
3,2018-01-28,3835.679915,3835.679915,1
4,2018-02-04,4276.198330,4276.198330,1


In [27]:
new_mean.to_excel('./가격데이터_증감비교_라벨링.xlsx',index=False)

### 월별 키워드 분석 맵핑

In [28]:
for i in range(1,13):
    stem_list=[]
    stem_list2=[]
    stem_list3=[]

    for text in df_18[df_18['월별']==month_list[i]]['article']:
        stem_list.append(okt.phrases(text))

    with open('./stem_list.txt','w',encoding='UTF-8') as f:
            f.write('\n'.join(map(str, stem_list)))

    f = open("./stem_list.txt", 'r',encoding='UTF-8')

    lines = f.readlines()

    for text in lines:
        malist=okt.pos(text, norm=True, stem=True)
        r=[]
        for word in malist:
            if not word[1] in ["Josa","Eomi","Punctuation", "Number",'\r','\t','Foreign','unknown']:
                r.append(word[0])
        rl=(" ".join(r)).strip()
        stem_list2.append(rl)
        print(rl)
    #     stem_list.append(okt.nouns(text))

    stem_list

    vect=CountVectorizer()

    vect.fit(stem_list2)
    globals()["new_df_{}".format(i)]=pd.DataFrame(vect.vocabulary_, index = [0])

    globals()["new_df_{}".format(i)]=globals()["new_df_{}".format(i)].T

    globals()["new_df_{}".format(i)].reset_index(inplace=True)

    globals()["new_df_{}".format(i)].columns=['word','frequency']

KeyError: '월별'

In [ ]:
new_df_12

In [ ]:
new_df_12_descending=new_df_12.sort_values(by=['frequency'], axis=0, ascending=False).reset_index(drop=True)

In [ ]:
new_df_12_descending

### 전월과 비교하여 공통된 단어 데이터프레임 만들기

In [ ]:
# 두 데이터프레임에서 추출한 단어들을 set 자료형으로 변환하여 비교
for i in range():
words_df1 = set(new_df_1['word'].values.flatten())
words_df2 = set(new_df_2['word'].values.flatten())
common_words = words_df1.intersection(words_df2)

common_words


In [ ]:
len(words_df2)